# Notebook Contents

- [Imports](#Imports)
- [Data](#Data)
- [Data Cleaning](#Data-Cleaning)
- [Preprocessing](#Preprocessing)
    - [Multicolinearity - VIF](#Multicolinearity---VIF)
- [Features](#Features)
- [Random Forest Modeling](#Random-Forest-Modeling)
    - [4-Seam](#Linear-Regression---4-Seam)
    - [Cutter](#Linear-Regression---Cutter)
    - [Sinker](#Linear-Regression---Sinker)
    - [Slider](#Linear-Regression---Slider)
    - [Curveball](#Linear-Regression---Curveball)
    - [Changeup](#Linear-Regression---Changeup)

# Imports

In [57]:
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans, k_means
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn import metrics
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV

import warnings
warnings.filterwarnings('ignore')

# Data

In [58]:
data = pd.read_csv('../data/model-pitches.csv', index_col = [0])
data.dropna(subset = ['pitch_type', 'velo', 'spin_rate', 'pfx_-x', 
                      'release_extension', 'delta_run_exp'], inplace = True)

pd.set_option('max_columns', None)
print(data.shape)
data.head(5)

(705403, 46)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,delta_run_exp,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,-0.073,R,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,Top,5,0,5,0,NaN,NaN,NaN,2
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,-0.027,R,foul,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,Top,5,0,5,0,NaN,NaN,NaN,2
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,-0.020,R,foul,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,Top,5,0,5,0,NaN,NaN,NaN,2
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0.016,R,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,Top,5,0,5,0,NaN,NaN,NaN,2
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,-0.189,L,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,Top,5,0,5,0,NaN,NaN,NaN,1


In [80]:
data.loc[19:25]

,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,delta_run_exp,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,home_runs,away_runs,runs,re,re_after,re_play
19,"Gsellman, Robert",R,FF,93.4,1999.0,241.0,8.16,10.68,21.402570,93.8,-1.71,5.85,6.3,54.22,1.61,-1.61,4.07,B,0,2,0.022,L,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,3,8,1,5,0,5,0,0,0,0,0,0,0,0,0.481,0.503,0.503
20,"Gsellman, Robert",R,CU,80.6,2660.0,62.0,-8.40,-5.64,33.002481,80.9,-1.64,5.72,6.2,54.31,0.92,-0.92,2.61,S,0,1,-0.052,L,called_strike,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,2,8,1,5,0,5,0,0,0,0,0,0,0,0,0.481,0.429,0.429
21,"Gsellman, Robert",R,SI,93.7,2028.0,234.0,16.44,6.24,21.643543,93.4,-1.69,5.76,6.2,54.31,0.68,-0.68,2.42,S,0,0,-0.038,L,foul,NaN,228.0,81.6,60.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,56,1,8,1,5,0,5,0,0,0,0,0,0,0,0,0.481,0.443,0.443
22,"Martin, Chris",R,CH,87.9,1160.0,234.0,7.68,1.80,13.196815,88.8,-2.98,6.23,6.7,53.81,0.36,-0.36,1.26,S,1,2,-0.073,L,swinging_strike,strikeout,NaN,NaN,NaN,NaN,NaN,NaN,0.0,1.0,0.0,0.0,55,5,8,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.025,0.025
23,"Martin, Chris",R,SL,84.9,2593.0,45.0,-6.60,-10.80,30.541814,84.9,-3.02,6.35,6.3,54.20,1.18,-1.18,1.25,S,1,2,0.000,L,foul,NaN,1.0,57.9,-52.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,4,8,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.098,0.098
24,"Martin, Chris",R,FF,95.3,2252.0,205.0,3.24,13.56,23.630640,96.4,-2.58,6.42,6.7,53.82,0.37,-0.37,2.75,S,1,1,-0.027,L,foul,NaN,200.0,72.1,24.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,3,8,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.071,0.071
25,"Martin, Chris",R,CH,89.2,1682.0,231.0,12.24,6.60,18.856502,89.9,-2.98,6.22,6.6,53.86,0.25,-0.25,0.70,B,0,1,0.014,L,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,55,2,8,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.112,0.112


In [59]:
re.head()

,on_1b,on_2b,on_3b,outs_when_up,re
0,0,0,0,0,0.481
1,1,0,0,0,0.859
2,0,1,0,0,1.100
3,1,1,0,0,1.437
4,0,0,1,0,1.350


# Data Cleaning

In [60]:
data['inning_topbot'] = data.inning_topbot.map({'Top': 0, 'Bot': 1})
data['on_1b'] = [1 if x > 1 else 0 for x in data['on_1b']]
data['on_2b'] = [1 if x > 1 else 0 for x in data['on_2b']]
data['on_3b'] = [1 if x > 1 else 0 for x in data['on_3b']]

data['home_runs'] = data['post_home_score'] - data['home_score']
data['away_runs'] = data['post_away_score'] - data['away_score']
data['runs'] = data['home_runs'] + data['away_runs']

In [61]:
zero_outs = data.loc[data['outs_when_up'] == 0]
print('0 outs:', zero_outs.shape)
one_out = data.loc[data['outs_when_up'] == 1]
print('1 out:', one_out.shape)
two_outs = data.loc[data['outs_when_up'] == 2]
print('2 outs:', two_outs.shape)

0 outs: (244451, 49)
1 out: (232568, 49)
2 outs: (228384, 49)


In [62]:
re = pd.read_csv('../data/run_expectancy_table.csv', index_col = [0])
data = pd.merge(data, re, how = 'left', on = ['on_1b', 'on_2b', 'on_3b', 'outs_when_up'])

data['re_after'] = data['delta_run_exp'] + data['re']
data['re_play'] = data['re_after'] + data['runs']

In [63]:
print(data.shape)
data.head()

(705403, 52)


,player_name,p_throws,pitch_type,velo,spin_rate,spin_axis,pfx_-x,pfx_z,bauer_units,effective_speed,release_pos_x,release_pos_z,release_extension,release_pos_y,plate_-x,plate_x,plate_z,type,balls,strikes,delta_run_exp,stand,description,events,hit_distance_sc,exit_velo,launch_angle,launch_speed_angle,xba,xwobacon,woba_value,woba_denom,babip_value,iso_value,at_bat_number,pitch_number,inning,inning_topbot,home_score,away_score,post_home_score,post_away_score,on_1b,on_2b,on_3b,outs_when_up,home_runs,away_runs,runs,re,re_after,re_play
0,"Smith, Will",L,FF,92.3,2330.0,148.0,-8.28,16.56,25.243770,92.8,1.40,6.80,6.5,54.03,0.69,-0.69,2.83,X,1,2,-0.073,R,hit_into_play,field_out,13.0,95.2,-13.0,2.0,0.174,0.158,0.0,1.0,0.0,0.0,61,4,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.025,0.025
1,"Smith, Will",L,SL,80.6,2254.0,315.0,9.24,5.76,27.965261,81.2,1.60,6.64,6.4,54.15,0.71,-0.71,2.62,S,1,1,-0.027,R,foul,NaN,108.0,75.3,75.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,3,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.071,0.071
2,"Smith, Will",L,CU,75.5,1940.0,328.0,7.80,-6.12,25.695364,75.2,1.46,6.88,6.2,54.34,0.04,-0.04,2.46,S,1,0,-0.020,R,foul,NaN,157.0,83.5,65.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,2,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.078,0.078
3,"Smith, Will",L,CU,75.0,2017.0,330.0,8.28,-8.28,26.893333,74.5,1.53,6.83,5.9,54.61,-2.10,2.10,3.89,B,0,0,0.016,R,ball,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,61,1,9,0,5,0,5,0,0,0,0,2,0,0,0,0.098,0.114,0.114
4,"Smith, Will",L,FF,91.2,2281.0,143.0,-7.56,15.36,25.010965,90.9,1.49,6.66,6.3,54.15,0.31,-0.31,2.80,X,1,0,-0.189,L,hit_into_play,field_out,9.0,93.3,-18.0,2.0,0.100,0.090,0.0,1.0,0.0,0.0,60,2,9,0,5,0,5,0,0,0,0,1,0,0,0,0.254,0.065,0.065


# Preprocessing

### Multicolinearity - VIF
**Independent Variables:** Velocity, Spin Rate, VB, HB, Release Extension, Horizontal Release Position, Vertical Release Position, Horizontal Plate Coords, Vertical Plate Coords

**Dependent Variable:** re_play


In [65]:
features = data[['velo', 'spin_rate', 'pfx_-x', 'pfx_z', 'release_extension', 
                 'release_pos_x', 'release_pos_z', 'plate_x', 'plate_z', 're_play',
                 'pitch_type', 'p_throws', 'stand']]
features_vif = features.select_dtypes([np.number])
vif_data = pd.DataFrame()
vif_data["feature"] = features_vif.columns

vif_data["VIF"] = [variance_inflation_factor(features_vif.values, i)
                   for i in range(len(features_vif.columns))]

vif_data.sort_values(by = 'VIF').head(10)

,feature,VIF
7,plate_x,1.110270
5,release_pos_x,1.454753
2,pfx_-x,1.466131
9,re_play,1.772727
3,pfx_z,3.115852
8,plate_z,7.449348
1,spin_rate,50.828327
6,release_pos_z,103.381165
4,release_extension,152.760973
0,velo,278.045447


# Features

In [66]:
ff = features.loc[features['pitch_type'] == 'FF']
fc = features.loc[features['pitch_type'] == 'FC']
fastball = ff.append(fc)
si = features.loc[features['pitch_type'] == 'SI']
fastball = fastball.append(si)
print('Fastball shape:', fastball.shape)
sl = features.loc[features['pitch_type'] == 'SL']
cu = features.loc[features['pitch_type'] == 'CU']
breaking_ball = sl.append(cu)
kc = features.loc[features['pitch_type'] == 'KC']
breaking_ball = breaking_ball.append(kc)
print('Breaking Ball:', breaking_ball.shape)
ch = features.loc[features['pitch_type'] == 'CH']
fs = features.loc[features['pitch_type'] == 'FS']
offspeed = ch.append(fs)
print('Off speed shape:', offspeed.shape)
rhp = features.loc[features['p_throws'] == 'R']
print('RHP shape:', rhp.shape)
lhp = features.loc[features['p_throws'] == 'L']
print('LHP shape:', lhp.shape)
rhp_rhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'R')]
print('RHP & RHH shape:', rhp_rhh.shape)
rhp_lhh = features.loc[(features['p_throws'] == 'R') & (features['stand'] == 'L')]
print('RHP & LHH shape:', rhp_lhh.shape)
lhp_rhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'R')]
print('LHP & RHH shape:', lhp_rhh.shape)
lhp_lhh = features.loc[(features['p_throws'] == 'L') & (features['stand'] == 'L')]
print('LHP & LHH shape:', lhp_lhh.shape)
rhp_fastball = fastball.loc[fastball['p_throws'] == 'R']
print('RHP Fastball shape:', rhp_fastball.shape)
lhp_fastball = fastball.loc[fastball['p_throws'] == 'L']
print('LHP Fastball shape:', lhp_fastball.shape)
rhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'R']
print('RHP Breaking Ball shape:', rhp_breaking_ball.shape)
lhp_breaking_ball = breaking_ball.loc[breaking_ball['p_throws'] == 'L']
print('LHP Breaking Ball shape:', lhp_breaking_ball.shape)
rhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'R']
print('RHP Offspeed shape:', rhp_offspeed.shape)
lhp_offspeed = offspeed.loc[offspeed['p_throws'] == 'L']
print('LHP Offspeed shape:', lhp_offspeed.shape)

Fastball shape: (406259, 13)
Breaking Ball: (207982, 13)
Off speed shape: (91162, 13)
RHP shape: (496498, 13)
LHP shape: (208905, 13)
RHP & RHH shape: (267548, 13)
RHP & LHH shape: (228950, 13)
LHP & RHH shape: (149824, 13)
LHP & LHH shape: (59081, 13)
RHP Fastball shape: (283224, 13)
LHP Fastball shape: (123035, 13)
RHP Breaking Ball shape: (152383, 13)
LHP Breaking Ball shape: (55599, 13)
RHP Offspeed shape: (60891, 13)
LHP Offspeed shape: (30271, 13)


# Linear Regression Model

## 4-Seam RHP

In [69]:
features_fastball_r = rhp_fastball.select_dtypes([np.number])
X = features_fastball_r.drop(columns = ['re_play'])
X = sm.add_constant(X)
y = features_fastball_r['re_play']

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 10)

ols_fastball_r = sm.OLS(y_train, X_train).fit()
pred_fastball_r = ols_fastball_r.predict(X)
fitted_vals_fastball_r = ols_fastball_r.fittedvalues
residuals_fastball_r = ols_fastball_r.resid

print('MSE:', round(metrics.mean_squared_error(y, pred_fastball_r), 4))
print('RMSE:', round(np.sqrt(metrics.mean_squared_error(y, pred_fastball_r)), 4))
print('MAE:', round(metrics.mean_absolute_error(y, pred_fastball_r), 4))
print(ols_fastball_r.summary())

MSE: 0.3269
RMSE: 0.5718
MAE: 0.3321
                            OLS Regression Results                            
Dep. Variable:                re_play   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     41.78
Date:                Sat, 05 Mar 2022   Prob (F-statistic):           2.04e-75
Time:                        23:14:44   Log-Likelihood:            -2.4354e+05
No. Observations:              283224   AIC:                         4.871e+05
Df Residuals:                  283214   BIC:                         4.872e+05
Df Model:                           9                                         
Covariance Type:            nonrobust                                         
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
c

# Random Forest Model

## 4-Seam RHP

In [85]:
features_fastball_r = rhp_fastball.select_dtypes([np.number])
X = features_fastball_r.drop(columns = ['re_play'])
y = features_fastball_r['re_play']

ss = StandardScaler()
X_fastball_scaled = ss.fit_transform(X)

X_train, X_test, y_train, y_test = train_test_split(X, y)

params = {
    'max_depth': [1, 2, 3, 4, 6, 8],
    'min_samples_leaf': [2, 3],
    'n_estimators': [175],
    'n_jobs': [-1]
}
    
gs = GridSearchCV(RandomForestRegressor(random_state = 6),
                  param_grid = params)
gs.fit(X_train, y_train)
print('Cross val score:', (gs.best_score_))
gs.best_params_

Cross val score: 0.00428549330890462


{'max_depth': 8, 'min_samples_leaf': 3, 'n_estimators': 175, 'n_jobs': -1}